In [23]:
from dotenv import load_dotenv
from pydantic_ai import Agent, NativeOutput
from rich import print as rich_print

load_dotenv()

True

In [10]:
naive_agent = Agent('openai:gpt-5-nano-2025-08-07')

In [11]:
result = await naive_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

- George Washington
- John Adams
- Thomas Jefferson
- James Madison
- James Monroe
- John Quincy Adams
- Andrew Jackson
- Martin Van Buren
- William Henry Harrison
- John Tyler

RunUsage(input_tokens=20, output_tokens=372, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 
'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, requests=1)

In [12]:
result = await naive_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

1. George Washington
2. John Adams
3. Thomas Jefferson
4. James Madison
5. James Monroe
6. John Quincy Adams
7. Andrew Jackson
8. Martin Van Buren
9. William Henry Harrison
10. John Tyler

RunUsage(input_tokens=20, output_tokens=446, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 
'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, requests=1)

In [24]:
from typing import List
from pydantic import BaseModel


class PresidentialTerm(BaseModel):
    name: str
    term_start_year: int
    term_end_year: int


class Presidents(BaseModel):
    presidents: List[PresidentialTerm]

structured_agent = Agent(
    'openai:gpt-5-nano-2025-08-07',
    output_type=NativeOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)

In [25]:
result = await structured_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=106, output_tokens=1749, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 
'reasoning_tokens': 1536, 'rejected_prediction_tokens': 0}, requests=1)

In [26]:
result = await structured_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=106, output_tokens=1941, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 
'reasoning_tokens': 1728, 'rejected_prediction_tokens': 0}, requests=1)

In [16]:
import ollama
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

OLLAMA_MODEL_NAME = "gpt-oss:20b"
OLLAMA_BASE_URL = "http://localhost:11434"

ollama.pull(OLLAMA_MODEL_NAME)

ollama_model = OpenAIChatModel(
    model_name=OLLAMA_MODEL_NAME,
    provider=OllamaProvider(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama"),
)

In [27]:
structured_ollama_agent = Agent[None, Presidents](
    ollama_model,
    output_type=NativeOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)


In [28]:
result = await structured_ollama_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=148, output_tokens=273, requests=1)